In [1]:
import numpy as np
import pyebsd  # https://github.com/arthursn/pyebsd
import quaternion  # https://pypi.org/project/numpy-quaternion/
from numba import jit, njit, prange

# Misorientation calculation using rotation matrices

In [2]:
@njit(parallel=True)
def misorientation_with_matrices(A, B, C):
    """
    Calculates all possible misorientation combinations between
    A and B using matrix operations
    
    Parameters
    ----------
    A: numpy ndarray(n, 3, 3)
        3D array listin n rotation matrices
    B: numpy ndarray(m, 3, 3)
        3D array listing m rotation matrices
    C: numpy ndarray(24, 3, 3)
        List of cubic symmetry operators
    
    Returns
    -------
    mis: numpy ndarray(n, m)
        Misorientation angle in radians
    """
    size_A, size_B, size_C = len(A), len(B), len(C)
    # Initialize empty mis array
    mis = np.zeros((size_A, size_B))
    # prange is used for parallelizing 
    for i in prange(size_A):
        for j in prange(size_B):
            trmax = -1.0
            for k in range(size_C):
                # T = C * A * B^T
                # tr = T_11 + T_22 + T_33
                tr = np.trace(C[k].dot(A[i]).dot(B[j].T))
                # Finds maximum value of trace (minimum angle)
                if tr > trmax:
                    trmax = tr
            # Due to rounding, trmax is slightly greater than 3, which is not allowed
            if trmax > 3.0:
                mis[i, j] = 0.0
            else:
                mis[i, j] = np.arccos((trmax - 1.0)/2.0)
    return mis

# Misorientation calculation using quaternions

In [3]:
@njit
def q_mult(q1, q2):
    """
    JIT compiled quaternion multiplication
    
    Parameters
    ----------
    q1, q1: numpy ndarray(4)
        Quaternions. First item is the scalar part
    
    Returns
    -------
    q : numpy ndarray(4)
        Quaternion product
    """
    q = np.zeros(4)
    q[0] = q1[0]*q2[0] - q1[1]*q2[1] - q1[2]*q2[2] - q1[3]*q2[3]
    q[1] = q1[0]*q2[1] + q1[1]*q2[0] + q1[2]*q2[3] - q1[3]*q2[2]
    q[2] = q1[0]*q2[2] - q1[1]*q2[3] + q1[2]*q2[0] + q1[3]*q2[1]
    q[3] = q1[0]*q2[3] + q1[1]*q2[2] - q1[2]*q2[1] + q1[3]*q2[0]
    return q

@njit(parallel=True)
def misorientation_with_quaternions(qA, qB, qC):
    """
    Calculates all possible misorientation combinations between
    A and B using quaternion operations
    
    Parameters
    ----------
    qA: numpy ndarray(n, 3, 3)
        3D array listing n quaternions
    qB: numpy ndarray(m, 3, 3)
        3D array listing m quaternions
    C: numpy ndarray(24, 3, 3)
        List of cubic symmetry operators
    
    Returns
    -------
    mis: numpy ndarray(n, m)
        Misorientation angle in radians
    """
    size_A, size_B, size_C = len(qA), len(qB), len(qC)
    # Initialize empty mis array
    mis = np.zeros((size_A, size_B))
    # prange is used for parallelizing 
    for i in prange(size_A):
        for j in prange(size_B):
            cos_half_theta_max = 0.0
            # This must not be parallelized
            for k in range(size_C):
                # To understand why this works, see the proof below
                # q = quaternion product between qC and qA
                # Then dot product between q and qB arrays and take absolute value
                # Result will be the cosine of half the misorientation angle
                cos_half_theta = np.abs(q_mult(qC[k], qA[i]).dot(qB[j]))
                # Finds maximum value of cos_half_theta (min theta)
                if cos_half_theta > cos_half_theta_max:
                    cos_half_theta_max = cos_half_theta
            if cos_half_theta_max > 1.0:
                mis[i, j] = 0.0
            else:
                mis[i, j] = 2 * np.arccos(cos_half_theta_max)
    return mis

In [4]:
def rotation_matrix_to_quaternion(M):
    """
    Converts rotation matrix to quaternion represented
    as array
    
    Parameters
    ----------
    M: numpy ndarray(..., 3, 3)
        Rotation matrix or list of rotation matrices
    
    Returns
    -------
    q: numpy ndarray(..., 4)
        Numpy array representing quaternion
    """
    return quaternion.as_float_array(quaternion.from_rotation_matrix(M))

# Performance test

In [7]:
# Generate random rotation matrices. You could load this from a 
# pyebsd ScanData object instead
N_A, N_B = 500, 400
A = pyebsd.euler_angles_to_rotation_matrix(*np.random.rand(3, N_A))
B = pyebsd.euler_angles_to_rotation_matrix(*np.random.rand(3, N_B))
# Casting to float is necessary
C = pyebsd.list_cubic_symmetry_operators().astype(float)

# A, B, C as quaternions
qA = rotation_matrix_to_quaternion(A)
qB = rotation_matrix_to_quaternion(B)
qC = rotation_matrix_to_quaternion(C)

%time mis = misorientation_with_matrices(A, B, C),
%time mis_q = misorientation_with_quaternions(qA, qB, qC)

# Check if all results match
np.count_nonzero(~np.isclose(mis, mis_q)) == 0

Calculating rotation matrices... 0.00 s
Calculating rotation matrices... 0.00 s
CPU times: user 20.7 s, sys: 22.2 s, total: 42.9 s
Wall time: 6.19 s
CPU times: user 4.33 s, sys: 132 ms, total: 4.46 s
Wall time: 638 ms


True

# Calculation of misorientation angle between two quaternions

Given two quarternions $q_1$ and $q_2$:

$$
q_1 = w_1 + x_1 i + y_1 j + z_1 k
$$

$$
q_2 = w_2 + x_2 i + y_2 j + z_2 k
$$

, the rotation $q_m$ between them is calculated:


$$
q_m = q_1 \cdot q_2^{-1} = w_m + x_m i + y_m j + z_m k
$$

$q_m$ can be converted to the angle $\theta_m$ - axis $(a_x, a_y, a_z)_m$ representation through the following formulas:

$$
\theta_m = 2 \arccos(w_m)
$$

$$
(a_x, a_y, a_z)_m = \frac{(x_m, y_m, z_m)}{\sqrt{1 - w_m^2}} = \frac{(x_m, y_m, z_m)}{\sqrt{x_m^2 + y_m^2 + z_m^2}}
$$

Notice that the axis is simply the normalized vector part of $q_m$.

$q_2^{-1}$ is simply:

$$
q_2^{-1} = w_2 - x_2 i - y_2 j - z_2 k
$$

We are only interested in the angle $\theta_m$, which only depends on the scalar part $w_m$. Only the diagonal of the quaternion multiplication table generates scalar terms:

|     |  *1*  |  *i*  |  *j*  |   *k* |
|-----|-------|-------|-------|-------|
| *1* | **1** |   i   |   j   |   k   |
| *i* |   i   |**-1** |   k   |  -j   |
| *j* |   j   |  -k   |**-1** |   i   |
| *k* |   k   |   j   |  -i   |**-1** |

Therefore:

$$
w_m = w_1 w_2 + (x_1 i) (-x_2 i) + (y_1 j) (-y_2 j) + (z_1 k) (-z_2 k) = w_1 w_2 + x_1 x_2 + y_1 y_2 + z_1 z_2
$$

And finally:

$$
\theta_m = 2 \arccos(w_1 w_2 + x_1 x_2 + y_1 y_2 + z_1 z_2)
$$